In [37]:
import pandas as pd
import numpy as np

# -------------------------------------------------
# STEP 1: Load original data
# -------------------------------------------------
df = pd.read_csv("ngs2020_pumf.csv")

# -------------------------------------------------
# STEP 2: Define variables to keep (original names)
# -------------------------------------------------
main_vars = [
    "JOBINCP",
    "STL_170A","STL_170B","STL_170C","STL_170D","STL_170E",
    "STL_170F","STL_170H","STL_170I","STL_170J","STL_170L","STLP170N"
]

control_vars = [
    "GRADAGEP","GENDER2","CTZSHIPP",
    "HLOSGRDP","PGMCIPAP",
    "REG_INST","BEF_160","PGM_P100",
    "PAR1GRD","PAR2GRD"
]

cols_to_keep = ["PUMFID"] + main_vars + control_vars
cols_to_keep = [c for c in cols_to_keep if c in df.columns]

# -------------------------------------------------
# STEP 3: Keep only selected columns
# -------------------------------------------------
df_selected_raw = df[cols_to_keep].copy()

df_selected_raw.to_csv("NGS2020_selected_raw.csv", index=False)
print("Saved file #1: NGS2020_selected_raw.csv")
print("Shape:", df_selected_raw.shape)

# -------------------------------------------------
# STEP 4: Rename columns
# -------------------------------------------------
rename_map = {
    "JOBINCP": "JobIncomeCategory",

    # Funding
    "STL_170A": "GovLoan",
    "STL_170B": "RESP",
    "STL_170C": "GovGrant",
    "STL_170D": "NonGovGrant",
    "STL_170E": "Scholarship",
    "STL_170F": "Savings",
    "STL_170H": "FamilySupport",
    "STL_170I": "BankLoan",
    "STL_170J": "CreditCard",
    "STL_170L": "EmployerFunding",
    "STLP170N": "OtherFunding",

    # Controls
    "GRADAGEP": "AgeGroup",
    "GENDER2": "Gender",
    "CTZSHIPP": "ImmigrationStatus",
    "HLOSGRDP": "EduLevel2020",
    "PGMCIPAP": "FieldOfStudy",
    "REG_INST": "ProvinceOfStudy",
    "BEF_160": "WorkBeforeProgram",
    "PGM_P100": "WorkPlacement",
    "PAR1GRD": "Parent1Edu_GradTime",
    "PAR2GRD": "Parent2Edu_GradTime"
}

df_selected_renamed = df_selected_raw.rename(
    columns={k: v for k, v in rename_map.items() if k in df_selected_raw.columns}
)

# -------------------------------------------------
# STEP 4.1: Recode JobIncomeCategory to representative numeric income
# -------------------------------------------------

# Convert first
df_selected_renamed["JobIncomeCategory"] = pd.to_numeric(
    df_selected_renamed["JobIncomeCategory"], errors="coerce"
)

# Map valid income categories to representative values
income_mid_map = {1: 13000, 2: 40000, 3: 60000, 4: 80000, 5: 120000}
df_selected_renamed["JobIncomeCategory"] = df_selected_renamed["JobIncomeCategory"].map(income_mid_map)

# Now apply special rules on the ORIGINAL codes

orig_income = pd.to_numeric(df_selected_raw["JOBINCP"], errors="coerce")

df_selected_renamed.loc[orig_income == 96, "JobIncomeCategory"] = 0
df_selected_renamed.loc[orig_income == 99, "JobIncomeCategory"] = np.nan





# -------------------------------------------------
# STEP 4.2: Clean funding variables (1 stays 1, otherwise 0)
# -------------------------------------------------
funding_cols = [
    "GovLoan", "RESP", "GovGrant", "NonGovGrant", "Scholarship",
    "Savings", "FamilySupport", "BankLoan", "CreditCard",
    "EmployerFunding", "OtherFunding"
]

# Keep only those that exist (safe)
funding_cols = [c for c in funding_cols if c in df_selected_renamed.columns]

for col in funding_cols:
    df_selected_renamed[col] = pd.to_numeric(df_selected_renamed[col], errors="coerce")
    df_selected_renamed[col] = np.where(df_selected_renamed[col] == 1, 1, 0)


# -------------------------------------------------
# STEP 5: Convert Gender → Male dummy
# -------------------------------------------------
df_selected_renamed["Male"] = df_selected_renamed["Gender"].apply(
    lambda x: 1 if x == 1 else 0
)
df_selected_renamed = df_selected_renamed.drop(columns=["Gender"])

# -------------------------------------------------
# STEP 6: Replace ProvinceOfStudy with Ontario / Quebec / WesternProv
# -------------------------------------------------
df_selected_renamed["Ontario"] = df_selected_renamed["ProvinceOfStudy"].apply(
    lambda x: 1 if x == 3 else 0
)

df_selected_renamed["Quebec"] = df_selected_renamed["ProvinceOfStudy"].apply(
    lambda x: 1 if x == 2 else 0
)

df_selected_renamed["WesternProv"] = df_selected_renamed["ProvinceOfStudy"].apply(
    lambda x: 1 if x == 4 else 0
)

# Remove the original province column
df_selected_renamed = df_selected_renamed.drop(columns=["ProvinceOfStudy"])

# -------------------------------------------------
# STEP 7: Replace AgeGroup with ageUnder25 / age25to29 / age30to39 / age40more
# -------------------------------------------------
df_selected_renamed["ageUnder25"] = df_selected_renamed["AgeGroup"].apply(
    lambda x: 1 if x == 1 else 0
)

df_selected_renamed["age25to29"] = df_selected_renamed["AgeGroup"].apply(
    lambda x: 1 if x == 2 else 0
)

df_selected_renamed["age30to39"] = df_selected_renamed["AgeGroup"].apply(
    lambda x: 1 if x == 3 else 0
)

df_selected_renamed["age40more"] = df_selected_renamed["AgeGroup"].apply(
    lambda x: 1 if x == 4 else 0
)

# Drop original AgeGroup column
df_selected_renamed = df_selected_renamed.drop(columns=["AgeGroup"])

# -------------------------------------------------
# STEP 8: Replace ImmigrationStatus with dummy variables
# -------------------------------------------------
df_selected_renamed["status_ctz_birth"] = df_selected_renamed["ImmigrationStatus"].apply(
    lambda x: 1 if x == 1 else 0
)

df_selected_renamed["status_ctz_nature"] = df_selected_renamed["ImmigrationStatus"].apply(
    lambda x: 1 if x == 2 else 0
)

df_selected_renamed["status_immigrant"] = df_selected_renamed["ImmigrationStatus"].apply(
    lambda x: 1 if x == 3 else 0
)

# Drop original variable
df_selected_renamed = df_selected_renamed.drop(columns=["ImmigrationStatus"])

# -------------------------------------------------
# STEP 9: Replace EduLevel2020 with dummy variables
# -------------------------------------------------
df_selected_renamed["edu_college"] = df_selected_renamed["EduLevel2020"].apply(
    lambda x: 1 if x == 1 else 0
)

df_selected_renamed["edu_bachelor"] = df_selected_renamed["EduLevel2020"].apply(
    lambda x: 1 if x == 2 else 0
)

df_selected_renamed["edu_graduate"] = df_selected_renamed["EduLevel2020"].apply(
    lambda x: 1 if x == 3 else 0
)

# Drop original variable
df_selected_renamed = df_selected_renamed.drop(columns=["EduLevel2020"])

# -------------------------------------------------
# STEP 10: Replace FieldOfStudy with dummy variables
# -------------------------------------------------

df_selected_renamed["fieldOfStudy_edu"] = df_selected_renamed["FieldOfStudy"].apply(
    lambda x: 1 if x == 1 else 0
)

df_selected_renamed["fieldOfStudy_human"] = df_selected_renamed["FieldOfStudy"].apply(
    lambda x: 1 if x == 2 else 0
)

df_selected_renamed["fieldOfStudy_social"] = df_selected_renamed["FieldOfStudy"].apply(
    lambda x: 1 if x == 4 else 0
)

df_selected_renamed["fieldOfStudy_business"] = df_selected_renamed["FieldOfStudy"].apply(
    lambda x: 1 if x == 5 else 0
)

df_selected_renamed["fieldOfStudy_lifeScience"] = df_selected_renamed["FieldOfStudy"].apply(
    lambda x: 1 if x == 6 else 0
)

df_selected_renamed["fieldOfStudy_math"] = df_selected_renamed["FieldOfStudy"].apply(
    lambda x: 1 if x == 7 else 0
)

df_selected_renamed["fieldOfStudy_engineering"] = df_selected_renamed["FieldOfStudy"].apply(
    lambda x: 1 if x == 8 else 0
)

df_selected_renamed["fieldOfStudy_health"] = df_selected_renamed["FieldOfStudy"].apply(
    lambda x: 1 if x == 9 else 0
)

# Drop original
df_selected_renamed = df_selected_renamed.drop(columns=["FieldOfStudy"])

# -------------------------------------------------
# STEP 11: Recode WorkBeforeProgram (BEF_160) into months
# -------------------------------------------------

# BEF_160 codes:
# 1 = none → 0 months
# 2 = <12 months → midpoint 6
# 3 = 12–23 months → midpoint 18
# 4 = 24–35 months → midpoint 30
# 5 = 36–47 months → midpoint 42
# 6 = 48+ months → set to 60 (chosen representative value)

months_map = {
    1: 0,
    2: 6,
    3: 18,
    4: 30,
    5: 42,
    6: 60
}

df_selected_renamed["WorkBeforeProgram"] = pd.to_numeric(
    df_selected_renamed["WorkBeforeProgram"], errors="coerce"
)

df_selected_renamed["WorkBeforeProgram"] = df_selected_renamed["WorkBeforeProgram"].map(months_map)

# -------------------------------------------------
# STEP X: Recode WorkPlacement (PGM_P100)
# -------------------------------------------------
df_selected_renamed["WorkPlacement"] = df_selected_renamed["WorkPlacement"].apply(
    lambda x: 1 if x == 1 else 0
)

# -------------------------------------------------
# STEP 12: Replace Parent1Edu_GradTime with dummy variables
# -------------------------------------------------

df_selected_renamed["Parent1Edu_higher"] = df_selected_renamed["Parent1Edu_GradTime"].apply(
    lambda x: 1 if x == 1 else 0
)

df_selected_renamed["Parent1Edu_same"] = df_selected_renamed["Parent1Edu_GradTime"].apply(
    lambda x: 1 if x == 2 else 0
)

df_selected_renamed["Parent1Edu_lower"] = df_selected_renamed["Parent1Edu_GradTime"].apply(
    lambda x: 1 if x == 3 else 0
)

# -------------------------------------------------
# STEP 13: Replace Parent2Edu_GradTime with dummy variables
# -------------------------------------------------

df_selected_renamed["Parent2Edu_higher"] = df_selected_renamed["Parent2Edu_GradTime"].apply(
    lambda x: 1 if x == 1 else 0
)

df_selected_renamed["Parent2Edu_same"] = df_selected_renamed["Parent2Edu_GradTime"].apply(
    lambda x: 1 if x == 2 else 0
)

df_selected_renamed["Parent2Edu_lower"] = df_selected_renamed["Parent2Edu_GradTime"].apply(
    lambda x: 1 if x == 3 else 0
)

# Drop original variables
df_selected_renamed = df_selected_renamed.drop(columns=["Parent1Edu_GradTime", "Parent2Edu_GradTime"])


# -------------------------------------------------
# STEP 14: Save final renamed file
# -------------------------------------------------
df_selected_renamed.to_csv("NGS2020_selected_renamed.csv", index=False)
print("Saved file #2: NGS2020_selected_renamed.csv")
print("Shape:", df_selected_renamed.shape)


Saved file #1: NGS2020_selected_raw.csv
Shape: (16138, 23)
Saved file #2: NGS2020_selected_renamed.csv
Shape: (16138, 43)
